## Load data

In [61]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn 

from torch.nn.functional import one_hot
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

In [7]:
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 
                'Weight', 'Acceleration', 'Model Year', 'Origin']

df = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
df = df.dropna()
df = df.reset_index(drop=True)
df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


## Split the data

In [15]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state=1)

In [16]:
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.404153,7.666909,9.0,17.5,23.0,29.0,46.6
Cylinders,313.0,5.402556,1.701506,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,189.512780,102.675646,68.0,104.0,140.0,260.0,455.0
Horsepower,313.0,102.929712,37.919046,46.0,75.0,92.0,120.0,230.0
Weight,313.0,2961.198083,848.602146,1613.0,2219.0,2755.0,3574.0,5140.0
Acceleration,313.0,15.704473,2.725399,8.5,14.0,15.5,17.3,24.8
Model Year,313.0,75.929712,3.675305,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.591054,0.807923,1.0,1.0,1.0,2.0,3.0


In [17]:
numeric_column_names = [ 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

## Preprocessing

In [29]:
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
for col in numeric_column_names:
    mean = train_stats.loc[col, 'mean']
    std = train_stats.loc[col, 'std']
    df_train_norm.loc[:, col] = (df_train_norm.loc[:, col]  - mean) / std
    df_test_norm.loc[:, col] = (df_test_norm.loc[:, col]  - mean) / std

df_train_norm = df_train_norm.reset_index(drop=True)
df_test_norm = df_test_norm.reset_index(drop=True)

df_test_norm.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,23.0,-0.824303,-0.677013,-0.156378,-0.536409,-0.441944,72,3
1,29.0,-0.824303,-0.901020,-0.736562,-0.931176,0.108434,75,3
2,32.4,-0.824303,-0.793886,-0.736562,-0.720241,0.401969,81,3
3,19.0,-0.824303,-0.667274,0.239201,-0.109825,-0.075025,73,2
4,38.0,-0.824303,-0.959456,-0.947537,-1.138576,0.181818,82,3


`Bucketing`

bucket = { <br>
0 if year < 73 <br>
1 if 73 ≤ year < 76 <br>
2 if 76 ≤ year < 79 <br>
3 if year ≥ 79 <br>

Create boundaries based on the buckets

In [34]:
boundaries = torch.tensor([73, 7, 79])

v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)


In [37]:
numeric_column_names.append('Model Year Bucketed')

In [38]:
df_test_norm.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin,Model Year Bucketed
0,23.0,-0.824303,-0.677013,-0.156378,-0.536409,-0.441944,72,3,2
1,29.0,-0.824303,-0.901020,-0.736562,-0.931176,0.108434,75,3,2
2,32.4,-0.824303,-0.793886,-0.736562,-0.720241,0.401969,81,3,3
3,19.0,-0.824303,-0.667274,0.239201,-0.109825,-0.075025,73,2,2
4,38.0,-0.824303,-0.959456,-0.947537,-1.138576,0.181818,82,3,3


One-hot Encoding into Categorical Feature

In [48]:
np.unique(df_train_norm['Origin'])

array([1, 2, 3], dtype=int64)

In [51]:
total_origin = len(set(df_train_norm['Origin']))
tensor_origin = torch.tensor(df_train_norm['Origin'].values)
encoded_origin = one_hot((tensor_origin) % total_origin)

x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, encoded_origin], 1).float()

tensor_origin = torch.tensor(df_test_norm['Origin'].values)
encoded_origin = one_hot((tensor_origin) % total_origin)

x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, encoded_origin], 1).float()

In [74]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

## Training the DNN Model

In [82]:
BATCH_SIZE = 8
NUM_EPOCHS = 200
LOG_EPOCHS = 20

Creating a Dataloader

In [84]:
torch.manual_seed(1)
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

In [85]:
all_layers = []
hidden_units_list = [2, 4]
input_size = df_train_norm.shape[1]

for hidden_unit in hidden_units_list:
    hidden_layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(hidden_layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

In [86]:
all_layers.append(nn.Linear(hidden_units_list[-1], 1))
all_layers

[Linear(in_features=9, out_features=2, bias=True),
 ReLU(),
 Linear(in_features=2, out_features=4, bias=True),
 ReLU(),
 Linear(in_features=4, out_features=1, bias=True)]

In [87]:
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Linear(in_features=9, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [88]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [89]:
torch.manual_seed(0)

for epoch in range(NUM_EPOCHS):
    loss_hist = 0
    for x_batch, y_batch in train_dl:
        preds = model(x_batch)[:, 0]
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_hist += loss.item()/len(train_dl)
    
    if epoch % LOG_EPOCHS == 0 :
        print(f'Epoch {epoch}:  Loss {round(loss.item(), 4)}')




Epoch 0:  Loss 5.4517
Epoch 20:  Loss 0.6217
Epoch 40:  Loss 6.0525
Epoch 60:  Loss 0.7024
Epoch 80:  Loss 0.0128
Epoch 100:  Loss 8.319
Epoch 120:  Loss 18.8336
Epoch 140:  Loss 1.3595
Epoch 160:  Loss 0.0336
Epoch 180:  Loss 2.9439


## Evaluate the Model

In [90]:
with torch.no_grad():
    preds = model(x_test)[:, 0]
    mse_loss = loss_fn(preds, y_test).item()
    mae_loss = nn.L1Loss()(preds, y_test).item()


print(f'MSE Loss: {mse_loss:.4f}')
print(f'MAE Loss: {mae_loss:.4f}')

MSE Loss: 14.2525
MAE Loss: 2.9643
